In [5]:
# Importa as bibliotecas necessárias do Selenium e outras auxiliares
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC

# Define uma classe para armazenar os dados do usuário
from dataclasses import dataclass

@dataclass
class Usuario:
    foto: str
    nome: str
    profissao: str
    email: str
    telefone: str
    perfil: str
    estado: str

# Inicializa o driver do navegador Chrome
driver = webdriver.Chrome()

# Abre a aba inicial da aplicação (Desafio 2)
driver.get('http://localhost:8000/#/Desafio/2')
driver.implicitly_wait(time_to_wait=15)  # Define um tempo de espera implícito para encontrar elementos

# Abre uma nova aba para o cadastro de usuários (Desafio 3)
driver.switch_to.new_window('tab')
driver.get('http://localhost:8000/#/Desafio/3')

# Mapeia as abas por nomes para facilitar a alternância
windows = {
    'busca': driver.window_handles[0],     # Aba de busca de usuários
    'cadastro': driver.window_handles[1]  # Aba de cadastro de usuários
}

# Inicia um loop para processar os usuários a serem cadastrados
while True:
    # Troca para a aba de cadastro
    driver.switch_to.window(window_name=windows['cadastro'])

    try:
        # Obtém o nome do próximo usuário a ser cadastrado
        usuario_busca = driver.find_element(By.ID, 'usuario').text
    except:
        # Se não encontrar mais usuários, encerra o loop
        break

    # Troca para a aba de busca
    driver.switch_to.window(window_name=windows['busca'])

    # Lista para armazenar os dados dos usuários encontrados
    data = []

    # Preenche o campo de busca com o nome do usuário e clica no botão de busca
    inp = driver.find_element(By.CSS_SELECTOR, 'main input')
    inp.clear()  # Limpa o campo antes de inserir um novo nome
    inp.send_keys(usuario_busca)  # Insere o nome do usuário

    # Aguarda o botão de busca ser clicável e o clica
    botao = driver.find_element(By.CSS_SELECTOR, 'main button')
    wait = WebDriverWait(driver=driver, timeout=60, poll_frequency=1)  # Espera explícita para o botão
    wait.until(EC.element_to_be_clickable(mark=botao))
    botao.click()

    # Aguarda os resultados da busca serem exibidos
    wait.until(EC.visibility_of_all_elements_located(locator=(By.CSS_SELECTOR, 'div.users-list > div > img')))

    # Obtém os elementos de cada usuário listado na busca
    users = driver.find_elements(By.CSS_SELECTOR, 'main section div.users-list > div')

    # Itera pelos usuários encontrados e coleta suas informações
    for user in users:
        foto = user.find_element(By.TAG_NAME, 'img')
        nome = user.find_element(By.TAG_NAME, 'h3')
        profissao = user.find_element(By.TAG_NAME, 'span')
        email = user.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(1)')
        telefone = user.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(2)')
        perfil = user.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(3)')
        estado = user.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(4)')

        # Cria uma instância da classe Usuario com os dados coletados
        dados_do_usuario = Usuario(
            foto=foto.get_attribute('src'),
            nome=nome.text,
            profissao=profissao.text,
            email=email.text[8:],  # Remove prefixos como "Email: "
            telefone=telefone.text[10:],  # Remove prefixos como "Telefone: "
            perfil=perfil.text[9:],  # Remove prefixos como "Perfil: "
            estado=estado.text[8:]  # Remove prefixos como "Estado: "
        )

        # Adiciona os dados do usuário à lista
        data.append(dados_do_usuario)

    # Troca novamente para a aba de cadastro para preencher os dados
    driver.switch_to.window(window_name=windows['cadastro'])

    # Localiza os campos do formulário de cadastro
    nome_cadastro = driver.find_element(By.NAME, 'nome')
    profissao_cadastro = driver.find_element(By.NAME, 'profissao')
    email_cadastro = driver.find_element(By.NAME, 'email')
    telefone_cadastro = driver.find_element(By.NAME, 'telefone')
    perfil_cadastro = driver.find_element(By.NAME, 'usuario')
    estado_cadastro = Select(driver.find_element(By.NAME, 'estado'))

    # Usa os dados do primeiro usuário encontrado para preencher o formulário
    dt = data[0]  # Dados do primeiro usuário

    # Preenche os campos do formulário
    nome_cadastro.clear()
    profissao_cadastro.clear()
    email_cadastro.clear()
    telefone_cadastro.clear()
    perfil_cadastro.clear()

    nome_cadastro.send_keys(dt.nome)
    profissao_cadastro.send_keys(dt.profissao)
    email_cadastro.send_keys(dt.email)
    telefone_cadastro.send_keys(dt.telefone)
    perfil_cadastro.send_keys(dt.perfil)
    estado_cadastro.select_by_visible_text(dt.estado)

    # Envia o formulário
    nome_cadastro.submit()

# Finaliza o driver após o término do processamento
#driver.quit()

In [11]:
data[0]


Usuario(foto='https://avatars.githubusercontent.com/u/94046082', nome='Natália Melo', profissao='Associado', email='natalia_melo41@example.net', telefone='(75) 9478-0714', perfil='natalia_melo', estado='Mato Grosso')